In [28]:
import os, ConfigParser, mediacloud, datetime

In [2]:
# load mediacloud and topic_id from config file
config = ConfigParser.ConfigParser()
config.read('app.config')

key = config.get('mediacloud','key')
topic_id = config.get('mediacloud', 'topic_id')

In [42]:
# instantiate mediacloud api
mc = mediacloud.api.MediaCloud(key)
mca = mediacloud.api.AdminMediaCloud(key)

# 1. Background Info

We're looking at the [US Presidential Election](https://topics.mediacloud.org/#/topics/1404/summary) topic in [Media Cloud](https://mediacloud.org/). That's topic ID #1404. This is a set of stories published between Apr 30, 2015 to Nov 7, 2016, queried on the names of the major presidential candidates. The topic is queried from the following media source sets:

* [US Top Online News](https://sources.mediacloud.org/#/collections/9139487)
* [US Top Digital Native News](https://sources.mediacloud.org/#/collections/9139458) 
* [US Regional Mainstream Media](https://sources.mediacloud.org/#/collections/2453107) 

The seed query is:

> +( fiorina ( scott and walker ) ( ben and carson ) trump ( cruz and -victor ) kasich rubio (jeb and bush) clinton sanders ) AND (+publish_date:[2016-09-30T00:00:00Z TO 2016-11-08T23:59:59Z]) AND ((tags_id_media:9139487 OR tags_id_media:9139458 OR tags_id_media:2453107 OR tags_id_stories:9139487 OR tags_id_stories:9139458 OR tags_id_stories:2453107))

I *think* this is the same dataset used for this CJR report, ["Breitbart-led right-wing media ecosystem altered broader media agenda"](https://www.cjr.org/analysis/breitbart-media-trump-harvard-study.php), but I'm not totally sure.

# 2. Network Structure

Run this section to request a gefx file representing the unweighted, directed network of media outlets in this dataset. Nodes represent different media outlets, edges represents inlinks and outlinks between outlets. 

In [4]:
# this api call takes a minute or two, but you should only need to do this once.

network = mc.topicMediaMap(topic_id)

with open('network.gefx', 'wb') as f:
    f.write(network)

In [7]:
# if you've already generated network.gefx, run this cell to import it

with open('network.gefx', 'r') as f:
    network = f.read()

In [ ]:
# figure out how to parse xml files here, maybe

# 3. Contagion Data

Now we want to see how a term/framing/quote *propagates* through our network. To do that, we need to search the stories in our topic (#1404) for mentions of a given term/framing/quote. Let's start with the term "alt-right".



In [29]:
# this is the query we're interested in. put the term(s) you want to search for here
term = "hillary AND clinton"

# this makes the actual query that we send to the api
query = '''{{~ topic:{0}}} AND "{1}"'''.format(topic_id, term)

# filter daterange
daterange = mc.publish_date_query( datetime.date (2015, 04, 30), datetime.date(2016,11,07) )

In [18]:
# here is a function that fetches every story from the topic that satisfies our query

def fetch_all_stories(solr_query, solr_filter=''):
    start = 0
    offset = 500
    all_stories = []
    page = 0

    while True:
        stories = mc.storyList(solr_query=solr_query, solr_filter=solr_filter, 
                               last_processed_stories_id=start, rows=offset)
        all_stories.extend(stories)
        
        if len(stories) < 1:
            break
        start = max([s['processed_stories_id'] for s in stories])
        page = page + 1

    return all_stories

In [32]:
mc.storyList(solr_query = query, rows = 10)

MCException: Error - got a HTTP status code of 500 with the message "Internal Server Error", body: {"error":"Error(s): Caught exception in MediaWords::Controller::Api::V2::Stories_Public->list \"transformed query is longer than max of 2000000 at lib/MediaWords/Solr/PseudoQueries.pm line 340.\n\tMediaWords::Solr::PseudoQueries::transform_query(\"( publish_date:[2015-04-30T00:00:00Z TO 2016-11-07T00:00:00Z}\"...) called at lib/MediaWords/Solr.pm line 264\n\tMediaWords::Solr::query_encoded_json(MediaWords::DB::HandlerProxy=HASH(0xcf974b8), HASH(0xc16d748), undef) called at lib/MediaWords/Solr.pm line 366\n\tMediaWords::Solr::query(MediaWords::DB::HandlerProxy=HASH(0xcf974b8), HASH(0xc16d748)) called at lib/MediaWords/Solr.pm line 641\n\tMediaWords::Solr::search_for_processed_stories_ids(MediaWords::DB::HandlerProxy=HASH(0xcf974b8), \"{~ topic:1404} AND \\\"hillary AND clinton\\\"\", ARRAY(0xc1e7af0), 0, 10, \"processed_stories_id\") called at lib/MediaWords/Controller/Api/V2/StoriesBase.p

In [33]:
query

'{~ topic:1404} AND "hillary AND clinton"'

In [46]:
mc.storyCount(solr_query = "{~ topic:1477} AND hillary AND clinton")

{u'count': 5458}

In [47]:
# for some reason storyCount is breaking if we query topic 1404... ugh
mc.storyCount(solr_query = "{~ topic:1404} AND hillary AND clinton")

MCException: Error - got a HTTP status code of 500 with the message "Internal Server Error", body: {"error":"Error(s): Caught exception in MediaWords::Controller::Api::V2::Stories_Public->count \"transformed query is longer than max of 2000000 at lib/MediaWords/Solr/PseudoQueries.pm line 340.\n\tMediaWords::Solr::PseudoQueries::transform_query(\"{~ topic:1404} AND hillary AND clinton\") called at lib/MediaWords/Solr.pm line 264\n\tMediaWords::Solr::query_encoded_json(MediaWords::DB::HandlerProxy=HASH(0x127c4608), HASH(0x1279ac48), MediaWords=HASH(0x127703d0)) called at lib/MediaWords/Solr.pm line 366\n\tMediaWords::Solr::query(MediaWords::DB::HandlerProxy=HASH(0x127c4608), HASH(0x1279ac48), MediaWords=HASH(0x127703d0)) called at lib/MediaWords/Controller/Api/V2/StoriesBase.pm line 461\n\tMediaWords::Controller::Api::V2::StoriesBase::count_GET(MediaWords::Controller::Api::V2::Stories_Public=HASH(0xab31608), MediaWords=HASH(0x127703d0)) called at /home/mediacloud/mediacloud/local/lib/per

In [48]:

network = mc.topicMediaMap(1477)

with open('network-1477.gefx', 'wb') as f:
    f.write(network)

In [55]:
mc.topicStoryList(1404, q="alternative AND right", limit=10)

{u'link_ids': {u'current': 23835, u'next': 23836},
 u'stories': [{u'bitly_click_count': 1,
   u'collect_date': u'2016-03-30 16:34:52.266809',
   u'date_is_reliable': 0,
   u'facebook_share_count': 30860,
   u'foci': [{u'focal_set_name': u'Retweet Partisanship Halves',
     u'foci_id': 581,
     u'name': u'Right Half',
     u'stories_id': 440271020},
    {u'focal_set_name': u'Retweet Partisanship Quintiles',
     u'foci_id': 579,
     u'name': u'Right',
     u'stories_id': 440271020},
    {u'focal_set_name': u'Mass Market Media',
     u'foci_id': 572,
     u'name': u'Mass Market Media',
     u'stories_id': 440271020}],
   u'full_text_rss': 0,
   u'guid': u'http://www.breitbart.com/tech/2016/03/29/an-establishment-conservatives-guide-to-the-alt-right/',
   u'inlink_count': 167,
   u'language': u'en',
   u'media_id': 19334,
   u'media_inlink_count': 96,
   u'media_name': u'Breitbart',
   u'normalized_tweet_count': None,
   u'outlink_count': 0,
   u'publish_date': u'2016-03-29 23:01:13',
 